In [4]:
conda install keras==2.3.1

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [1]:
import keras_preprocessing
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint


#for accuracy and loss graph
import matplotlib.pyplot as plt

import tensorflow
print(keras.__version__)
print(tensorflow.__version__)
#ResNet, short for Residual Networks is a classic neural network used as a backbone for many computer vision tasks.
#resnet152V2

Using TensorFlow backend.


2.3.1
2.0.0


In [2]:
train_data_path = "/content/drive/MyDrive/Colab Notebooks/mask detection/dataset/train"
validation_data_path = "/content/drive/MyDrive/Colab Notebooks/mask detection/dataset/valid"

In [3]:
# check for augmented images
def plotImages(images_arr):
  fig, axes = plt.subplots(1, 5, figsize=(20, 20))
  axes = axes.flatten()
  for img, ax in zip(images_arr, axes):
    ax.imshow(img)
  plt.tight_layout()
  plt.show()

In [4]:
# this is the augmentation configuration we will use for trainig
#it generates more images using the parameters defined below
# deep learning requires a lot of datasets for prediction model to work well
#use augmentation configuration to generate more images using datasets, and use 
#the parameters as shown below, augemnt images based on parameters below(zoom, crop, resize etc)
training_datagen = ImageDataGenerator(rescale=1./255,
                                      rotation_range=40,
                                      width_shift_range=0.2,
                                      height_shift_range=0.2,
                                      shear_range=0.2,
                                      zoom_range=0.2,
                                      horizontal_flip=True,
                                      fill_mode='nearest')

In [5]:
#this data will read images from dataset(from train data) 
# and generate batches of augmented data indefinitely
training_data = training_datagen.flow_from_directory(train_data_path, #target directory we want to use
                                                     target_size=(200,200), #resize the images to this
                                                     batch_size= 128,
                                                     class_mode= 'binary') #since we use binary_crossentropy loss, we need binary

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/content/drive/MyDrive/Colab Notebooks/mask detection/dataset/train'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
training_data.class_indices

In [ ]:
#rescaling
#augmentation configuration to be used for validation
valid_datagen = ImageDataGenerator(rescale=1./255)
#we now apply similar generator for the validation data(above we did it for training data)
validation_data = valid_datagen.flow_from_directory(validation_data_path, #target directory we want to use
                                                     target_size=(200,200), #resize the images to this
                                                     batch_size= 128,
                                                     class_mode= 'binary') #since we use binary_crossentropy loss, we need binary


In [3]:
validation_data.class_indices

NameError: name 'validation_data' is not defined

In [ ]:
#check to see augmented images
images=[training_data[0][0][0] for i in range(5)]
plotImages(images)

In [ ]:
#save the best model among all the epochs
model_path = '/content/drive/MyDrive/Colab Notebooks/modelsJ/model_jiya.h5'
checkpoint = ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
#building cnn model
cnn_model = keras.models.Sequential([
                                     keras.layers.Conv2D(filters=32, kernel_size=5, input_shape=[200,200,3]),
                                     keras.layers.MaxPooling2D(pool_size=(4,4)),
                                     keras.layers.Conv2D(filters=64, kernel_size=4),
                                     keras.layers.MaxPooling2D(pool_size=(3,3)),
                                     keras.layers.Conv2D(filters=128, kernel_size=3),
                                     keras.layers.MaxPooling2D(pool_size=(2,2)),
                                     keras.layers.Conv2D(filters=256, kernel_size=2),
                                     keras.layers.MaxPooling2D(pool_size=(2,2)), #pool size and kernel sizes are changing

                                     keras.layers.Dropout(0.5),
                                     keras.layers.Flatten(), #neural network building, make filters into single array
                                     keras.layers.Dense(units=128, activation='relu'), #input layers
                                     keras.layers.Dropout(0.1),
                                     keras.layers.Dense(units=256, activation='relu'),
                                     keras.layers.Dropout(0.25),
                                     keras.layers.Dense(units=2, activation='softmax') #output layer
])

In [ ]:
#compile cnn model
cnn_model.compile(optimizer = Adam(lr=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy']) #lr(learning rate is 0.001, it's slow but very accurate)
#cnn_model.compile(optimizer = Adam(lr=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])#smaller the learning rate, more accurate

In [ ]:
#train cnn model
history = cnn_model.fit(training_data,
                        epochs=50,
                        verbose=1,
                        validation_data= validation_data,
                        callbacks=callbacks_list) #time start 14.25

In [ ]:
cnn_model.save('/content/drive/MyDrive/Colab Notebooks/modelsJ/model_last_jiya.h5')

In [ ]:
#train cnn model again, it will start with epoch 51 to 100 (basically training another 50 epochs to a total of 100 epochs)
#more epochs will improve accuracy and prediction will be faster too
history = cnn_model.fit(training_data,
                        epochs=50,
                        verbose=1,
                        validation_data= validation_data,
                        callbacks=callbacks_list) #time start 14.25

In [ ]:
cnn_model.save('/content/drive/MyDrive/Colab Notebooks/modelsJ/model_last_jiya.h5')

In [ ]:
import matplotlib.pyplot as plt
#plot the loss
plt.plot(cnn_model.history['loss'], label='train loss')
plt.plot(cnn_model.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

#plot the accuracy
plt.plot(cnn_model.history['accuracy'], label='train acc')
plt.plot(cnn_model.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')